In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import tensorflow.examples.tutorials.mnist.input_data as input_data
mnist=input_data.read_data_sets('MNIST_data',one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
n_input=784
n_output=10
stddev=0.1
weights={
    'wc1':tf.Variable(tf.random_normal([3,3,1,64],stddev=stddev)),
    'wc2':tf.Variable(tf.random_normal([3,3,64,128],stddev=stddev)),
    'wd1':tf.Variable(tf.random_normal([7*7*128,1024],stddev=stddev)),
    'wd2':tf.Variable(tf.random_normal([1024,10],stddev=stddev))
}
biases={
    'bc1':tf.Variable(tf.zeros([64])),
    'bc2':tf.Variable(tf.zeros([128])),
    'bd1':tf.Variable(tf.zeros([1024])),
    'bd2':tf.Variable(tf.zeros([n_output]))
}

In [3]:
def conv(_input,_w,_b,_keepratio):
    #input
    _input_r=tf.reshape(_input,shape=[-1,28,28,1])
    conv1=tf.nn.conv2d(_input_r,weights['wc1'],strides=[1,1,1,1],padding="SAME")
    conv1=tf.nn.relu(tf.nn.bias_add(conv1,biases['bc1']))
    pool1=tf.nn.max_pool(conv1,ksize=[1,2,2,1],strides=[1,2,2,1],padding="SAME")
    pool_dr1=tf.nn.dropout(pool1,_keepratio)
    conv2=tf.nn.conv2d(pool_dr1,weights['wc2'],strides=[1,1,1,1],padding="SAME")
    conv2=tf.nn.relu(tf.nn.bias_add(conv2,biases['bc2']))
    pool2=tf.nn.max_pool(conv2,ksize=[1,2,2,1],strides=[1,2,2,1],padding="SAME")
    pool_dr2=tf.nn.dropout(pool2,_keepratio)
    print pool_dr2.shape
    print _w['wd1'].get_shape().as_list()[0]/784
    _dense1=tf.reshape(pool_dr2,[-1,_w['wd1'].get_shape().as_list()[0]])
    print _dense1.shape
    _fc1=tf.nn.relu(tf.add(tf.matmul(_dense1,_w['wd1']),_b['bd1']))
    fc_dr1=tf.nn.dropout(_fc1,_keepratio)
    _out=tf.add(tf.matmul(fc_dr1,_w['wd2']),_b['bd2'])
    out={
        'input_r':_input_r,'conv1':conv1,'pool1':pool1,
        'pool_dr1':pool_dr1,'conv2':conv2,'pool2':pool_dr2,
        '_dense1':_dense1,'_fc1':_fc1,'fc_dr1':fc_dr1,'_out':_out
    }
    return out

In [4]:
print "CNN OK"

CNN OK


In [5]:
x = tf.placeholder(tf.float32, [None, n_input])
y = tf.placeholder(tf.float32, [None, n_output])

In [6]:
keepratio = tf.placeholder(tf.float32)

In [19]:
pre = conv(x, weights, biases, keepratio)['_out']

(?, 7, 7, 128)
8
(?, 6272)


In [20]:
print "OOOOO"

OOOOO


In [21]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pre,labels=y))

In [22]:
optm = tf.train.AdamOptimizer(0.01).minimize(cost)

In [23]:
_corr = tf.equal(tf.argmax(pre, 1), tf.argmax(y, 1))
accr = tf.reduce_mean(tf.cast(_corr, tf.float32))

In [24]:
init = tf.global_variables_initializer()

In [25]:
print "GRAPH OK"

GRAPH OK


In [27]:
train_epochs=20
batch_size=32
display_step=3
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(train_epochs):
        avg_cost=0.0
        total_batch=int(mnist.train.num_examples/batch_size)
        for i in range(
                total_batch):
            batch_xs,batch_ys=mnist.train.next_batch(batch_size)
            sess.run(optm,feed_dict={x:batch_xs,y:batch_ys,keepratio:0.8})
            avg_cost+=sess.run(cost,feed_dict={x:batch_xs,y:batch_ys,keepratio:1.})/total_batch
        if epoch%display_step==0:
            print "epoch:%03d/%03d   cost   :%.9f"%(epoch,train_epochs,avg_cost)
            trainacc=sess.run(accr,feed_dict={x:batch_xs,y:batch_ys,keepratio:1.})
            print " Train accuracy: %.3f" % trainacc
            feeds={x:mnist.test.images,y:mnist.test.labels}
            test_acc=sess.run(accr,feed_dict=feeds)
            print "Test accuracy: %.3f"%test_acc


KeyboardInterrupt: 